In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1" # Força o Keras antigo
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" # Limpa mensagens de log desnecessárias
import tensorflow as tf
import shutil
import numpy as np
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity



# Configuração específica para sua RTX 4050 (Laptop GPUs têm gerenciamento de energia rigoroso)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
      
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(">> GPU RTX 4050 configurada com sucesso!")
    except RuntimeError as e:
        print(e)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU Detectada e Ativa: {gpus}")
    except RuntimeError as e:
        print(e)

# --- CONFIGURAÇÕES DE CAMINHOS ---
PASTA_CONHECIDOS = r"/home/maykollrocha/Base_dados/imagens_Conhecidas"
PASTA_ORIGEM = r"/home/maykollrocha/Base_dados/Imagens_Para_Analisar"
PASTA_SAIDA_BASE = r"/home/maykollrocha/Base_dados/Rostos Detectados"
PASTA_NAO_ENCONTRADOS = r"/home/maykollrocha/Base_dados/nao encontrados"

# --- CONFIGURAÇÕES DO MODELO ---
MODELO = "ArcFace" 
DETECTOR = "retinaface" 
THRESHOLD = 0.40

>> GPU RTX 4050 configurada com sucesso!
GPU Detectada e Ativa: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
def organizar_fotos_v2():
    
# 1. Pergunta inicial ao usuário
    pergunta = input("Gostaria de fazer uma separação otimizada? (Imagens encontradas não serão analisadas novamente) [S/N]: ").strip().upper()
    otimizado = True if pergunta == "S" else False

    # Cria as pastas base se não existirem
    os.makedirs(PASTA_SAIDA_BASE, exist_ok=True)
    os.makedirs(PASTA_NAO_ENCONTRADOS, exist_ok=True)

    # Lista as imagens conhecidas (referências), caso tenha outros formatos, pode ser necessário ajustar a extensão
    fotos_conhecidas = [f for f in os.listdir(PASTA_CONHECIDOS) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Lista as imagens para analisar
    Imagens_Para_Analisar = [f for f in os.listdir(PASTA_ORIGEM) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Conjunto para rastrear o que já foi encontrado (para a pasta "não encontrados" e otimização)
    encontrados_geral = set()

    print(f"\n--- Iniciando Processamento ---")
    print(f"Modo Otimizado: {'ATIVADO' if otimizado else 'DESATIVADO'}")
    print(f"Pessoas conhecidas: {len(fotos_conhecidas)}")
    print(f"Total de imagens para analisar: {len(Imagens_Para_Analisar)}\n")

    # 1. Mapear Embeddings das Referências (Fotos conhecidas)
    referencias = {}
    print(">> Extraindo características das fotos de referência...")
    for foto_ref in fotos_conhecidas:
        nome_pessoa = os.path.splitext(foto_ref)[0]
        caminho_ref = os.path.join(PASTA_CONHECIDOS, foto_ref)
        try:
            # Extrai o vetor da face (embedding)
            embedding = DeepFace.represent(img_path=caminho_ref, model_name=MODELO, 
                                           detector_backend=DETECTOR, enforce_detection=False)[0]["embedding"]
            referencias[nome_pessoa] = embedding
        except Exception as e:
            print(f"Erro na referência {foto_ref}: {e}")

    # 2. Mapear Embeddings das Fotos de Origem (Uma única vez!)
    print(f">> Analisando {len(Imagens_Para_Analisar)} fotos da origem...")
    embeddings_origem = []
    for img in Imagens_Para_Analisar:
        caminho_alvo = os.path.join(PASTA_ORIGEM, img)
        try:
            # Aqui a GPU brilha: processando o lote
            rep = DeepFace.represent(img_path=caminho_alvo, model_name=MODELO, 
                                     detector_backend=DETECTOR, enforce_detection=False)
            # Guardamos o nome do arquivo e o vetor
            embeddings_origem.append({"arquivo": img, "embedding": rep[0]["embedding"]})
        except:
            continue

    # 3. Comparação Matemática (Ultra Rápida)
    encontrados_geral = set()
    for nome_pessoa, vector_ref in referencias.items():
        print(f">> Cruzando dados para: {nome_pessoa}")
        pasta_destino = os.path.join(PASTA_SAIDA_BASE, nome_pessoa)
        os.makedirs(pasta_destino, exist_ok=True)
        
        vector_ref = np.array(vector_ref).reshape(1, -1)

        for item in embeddings_origem:
            arquivo = item["arquivo"]
            # Se otimizado e já achou, pula (opcional, conforme sua lógica)
            if otimizado and arquivo in encontrados_geral:
                continue
                
            vector_alvo = np.array(item["embedding"]).reshape(1, -1)
            
            # Cálculo de similaridade de cosseno
            distancia = 1 - cosine_similarity(vector_ref, vector_alvo)[0][0]

            if distancia <= THRESHOLD:
                shutil.copy(os.path.join(PASTA_ORIGEM, arquivo), os.path.join(pasta_destino, arquivo))
                encontrados_geral.add(arquivo)
if __name__ == "__main__":
    organizar_fotos_v2()


--- Iniciando Processamento ---
Modo Otimizado: ATIVADO
Pessoas conhecidas: 10
Total de imagens para analisar: 212

>> Extraindo características das fotos de referência...


I0000 00:00:1772050049.668218    1214 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3536 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


>> Analisando 212 fotos da origem...
>> Cruzando dados para: user02
>> Cruzando dados para: user10
>> Cruzando dados para: user04
>> Cruzando dados para: user03
>> Cruzando dados para: user01
>> Cruzando dados para: user05
>> Cruzando dados para: user09
>> Cruzando dados para: user07
>> Cruzando dados para: user06
>> Cruzando dados para: user08
